## Source Information
---
**Author**:

**Last Updated Date**: October 01, 2024 by Gloria Seo

**Resources**: https://medium.com/analytics-vidhya/sequence-model-time-series-prediction-using-tensorflow-2-0-665257beb25f

---

## Goal
This Jupyter Notebook demonstrates how to build a simple time series forecasting model using TensorFlow. Specifically, we are working with historical data of the exchange rate between USD and INR (Indian Rupee), which we will download as a CSV file. The goal is to predict future values based on past trends using a sequential deep learning model.



# Downloading and Preparing the Data

We download a CSV file containing USD to INR exchange rates from the internet. This data will be stored locally and used to train the forecasting model.

In [1]:
# Download the CSV file using wget
!wget --no-check-certificate https://raw.githubusercontent.com/satishnaidu/mldata/master/timeseries/USD_INR.csv -O /tmp/USD_INR.csv

# Define the file path
file_path = '/tmp/USD_INR.csv'


wget: /cm/shared/apps/spack/cpu/opt/spack/linux-centos8-zen/gcc-8.3.1/anaconda3-2020.11-da3i7hmt6bdqbmuzq6pyt7kbm47wyrjp/lib/libuuid.so.1: no version information available (required by wget)
--2025-03-31 21:54:56--  https://raw.githubusercontent.com/satishnaidu/mldata/master/timeseries/USD_INR.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 130380 (127K) [text/plain]
Saving to: ‘/tmp/USD_INR.csv’

/tmp/USD_INR.csv    100%[===================>] 127.32K  --.-KB/s    in 0.005s  

2025-03-31 21:54:56 (24.8 MB/s) - ‘/tmp/USD_INR.csv’ saved [130380/130380]



## Required Modules for the Jupyter Notebook
Before running the notebook, make sure to load these modules.

**Module: tensorflow, numpy, csv, matplotlib** 


In [2]:
import tensorflow as tf
import numpy as np
import csv
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'tensorflow'

## Plotting the Data
To better understand the dataset, we will first visualize the exchange rates over time. The CSV file is read, and the relevant columns (date and exchange rate) are extracted for plotting. A function plot_series() is used to neatly display the time series data.

In [ ]:
def plot_series(time, series, format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.grid(True)

In [ ]:
time_step = []
inr_conversion = []
with open(file_path, newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        inr_conversion.append(float(row[2]))
        time_step.append(int(row[0]))
        
print(inr_conversion[len(time_step)-1])
series = np.array(inr_conversion)
time = np.array(time_step)
plt.figure(figsize=(10, 6))
plot_series(time, series)

This code visualizes the USD-INR exchange rate over time, giving a sense of how the currency value fluctuates.

## Splitting the Data

Next, we split the data into two parts:

1. Training Data: Used to train the model.
2. Validation Data: Used to test how well the model performs on unseen data.


In [ ]:
split_time = 2000
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]
window_size = 60
batch_size = 100
shuffle_buffer_size = 1000

## Creating a Windowed Dataset

A time series prediction model learns better when we provide it with small "windows" of data points. The windowed_dataset() function helps create batches of sequential data for training.

In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

## Building the Model
The model is built using a sequential structure that includes:

Convolutional Layers: Extracts patterns from the time series data.

LSTM (Long Short-Term Memory) Layers: Excellent for learning from sequential data.

Dense Layers: Final layers for producing the output predictions.

In [ ]:
def model_forecast(model, series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(32).prefetch(1)
    forecast = model.predict(ds)
    return forecasttf.random.set_seed(51)

np.random.seed(51)
train_set = windowed_dataset(x_train, window_size=120,
                             batch_size=100,
                             shuffle_buffer=shuffle_buffer_size)

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(filters=60, kernel_size=5,
                         strides=1,padding="causal",
                         activation="relu",input_shape=[None, 1]),
  tf.keras.layers.LSTM(60, return_sequences=True),
  tf.keras.layers.LSTM(60, return_sequences=True),
  tf.keras.layers.Dense(30, activation="relu"),
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(1),
  tf.keras.layers.Lambda(lambda x: x * 400)
])

optimizer = tf.keras.optimizers.SGD(learning_rate=51e-7, momentum=0.9)

model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])
history = model.fit(train_set,epochs=500)

## Last Updated Date and Resources
---
**Last Updated Date**: October 01, 2024

**Resources**: https://medium.com/analytics-vidhya/sequence-model-time-series-prediction-using-tensorflow-2-0-665257beb25f

---

## Submit Ticket
If you find anything that needs to be changed, edited, or if you would like to provide feedback or contribute to the notebook, please submit a ticket by contacting us at:

Email: consult@sdsc.edu

We appreciate your input and will review your suggestions promptly!